In [ ]:
import pandas as pd

# Load the CSV file
file_path = "reduced_output.csv"
df = pd.read_csv(file_path)

# Display basic info and the first few rows
df_info = df.info()
df_head = df.head()

df_info, df_head

In [61]:
import pandas as pd

def reduce_csv_to_1000_rows(input_csv, output_csv):
    # Read the CSV file
    df = pd.read_csv(input_csv)

    # If the dataset has more than 1000 rows, randomly sample 1000 rows
    if len(df) > 1000:
        df_reduced = df.sample(n=7100, random_state=42)  # random_state for reproducibility
    else:
        df_reduced = df  # if there are fewer than 1000 rows, keep all rows

    # Save the reduced dataset to a new CSV file
    df_reduced[["Text", "Subreddit", "Category"]].to_csv(output_csv, index=False)

    print(f"Data reduced and saved to {output_csv}")

# Example usage
input_csv = 'C:/personal/Code/S6/NLP/Slang Classification/Datas/merge_all_text.csv'  # Replace with your input CSV file path
output_csv = 'reduced_output.csv'  # Replace with your desired output file path
reduce_csv_to_1000_rows(input_csv, output_csv)


Data reduced and saved to reduced_output.csv


In [ ]:
import pandas as pd
import ast
from datetime import datetime, timezone

df = pd.read_csv('Original Data.csv')  # Replace 'your_file.csv' with your actual file path

# Create DataFrame
post_data = pd.DataFrame(df)

# Step 1: Combine Title, Text, and Comments Data into separate rows

# Convert Comments Data from string format to list
post_data['Comments Data'] = post_data['Comments Data'].apply(ast.literal_eval)

# Prepare the DataFrame for expanding to multiple rows
expanded_data = []

# Add the Title and Text as separate rows
for _, row in post_data.iterrows():
    # Add the title as a row
    expanded_data.append({'Text': row['Title'], 'Category': 'No Slang', 'Date': datetime.fromtimestamp(row['Timestamp'], tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'), 'Upvotes': row['Upvotes'], 'Subreddit': row['Subreddit']})
    
    # Add the text (if available)
    if row['Text']:
        expanded_data.append({'Text': row['Text'], 'Category': 'No Slang', 'Date': datetime.fromtimestamp(row['Timestamp'], tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'), 'Upvotes': row['Upvotes'], 'Subreddit': row['Subreddit']})
    
    # Add each comment as a separate row
    for comment in row['Comments Data']:
        expanded_data.append({'Text': comment[0], 'Category': 'No Slang', 'Date': datetime.fromtimestamp(comment[2], tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'), 'Upvotes': comment[1], 'Subreddit': row['Subreddit']})

# Create the expanded DataFrame
expanded_df = pd.DataFrame(expanded_data)

expanded_df.to_csv('merge_all_text.csv', index=False)  # Saves the cleaned data to a new file

print(expanded_df)



In [ ]:
import pandas as pd

# Load the CSV file
file_path = "C:/personal/Code/S6/NLP/Slang Classification/Datas/reduced_output.csv"
df = pd.read_csv(file_path)

# Display basic info and the first few rows
df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       6920 non-null   object
 1   Subreddit  6920 non-null   object
 2   Category   6920 non-null   object
dtypes: object(3)
memory usage: 162.3+ KB


(None,
                                                 Text  Subreddit  \
 0  this man is a cut above the rest outdoor or in...     soccer   
 1                                       why not four      funny   
 2                                          maga baby  AskReddit   
 3  i doubt even chatgpt would come up with a stor...     gaming   
 4  i feel thats kinda true but at the same time a...     soccer   
 
           Category  
 0         No Slang  
 1         No Slang  
 2  Offensive Slang  
 3         No Slang  
 4         No Slang  )

In [ ]:
df[['Text', 'Category', 'Subreddit']].to_csv('target_data_2.csv', index=False)

In [3]:
import pandas as pd

# Load the CSV file
file_path = "C:/personal/Code/S6/NLP/Slang Classification/Datas/Labelled.csv"
df = pd.read_csv(file_path)

# Display basic info and the first few rows
df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       6920 non-null   object
 1   Subreddit  6920 non-null   object
 2   Category   6920 non-null   object
dtypes: object(3)
memory usage: 162.3+ KB


(None,
                                                 Text  Subreddit  \
 0  this man is a cut above the rest outdoor or in...     soccer   
 1                                       why not four      funny   
 2                                          maga baby  AskReddit   
 3  i doubt even chatgpt would come up with a stor...     gaming   
 4  i feel thats kinda true but at the same time a...     soccer   
 
           Category  
 0         No Slang  
 1         No Slang  
 2  Offensive Slang  
 3         No Slang  
 4         No Slang  )

In [4]:
import numpy as np
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Salin dataframe
df_cleaned = df.copy()

# --- Step 1: Data Cleaning ---
df_cleaned = df_cleaned.dropna(subset=['Text'])
df_cleaned = df_cleaned.drop_duplicates()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_cleaned["Text"] = df_cleaned["Text"].apply(clean_text)
# Remove parentheses from Category column
df_cleaned["Category"] = df_cleaned["Category"].str.replace(r"\([^)]*\)", "", regex=True).str.strip()

# # --- Step 2: Filter by Text Length (Word Count) ---
df_cleaned["TextLength"] = df_cleaned["Text"].apply(lambda x: len(x.split()))

# Filter out texts with fewer than 5 words
df_cleaned = df_cleaned[df_cleaned["TextLength"] >= 5]

# Remove outliers based on TextLength distribution
lower_bound = df_cleaned["TextLength"].quantile(0.01)
upper_bound = df_cleaned["TextLength"].quantile(0.99)

df_cleaned = df_cleaned[(df_cleaned["TextLength"] >= lower_bound) & 
                        (df_cleaned["TextLength"] <= upper_bound)]

df_cleaned[['Text', 'Category', 'Subreddit']].to_csv('Data_cleaned.csv', index=False)  # Saves the cleaned data to a new file
df_cleaned.info()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andif\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andif\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
Index: 5636 entries, 0 to 6919
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Text        5636 non-null   object
 1   Subreddit   5636 non-null   object
 2   Category    5636 non-null   object
 3   TextLength  5636 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 220.2+ KB


In [5]:
# --- Step 3: Tokenization and Stopwords Removal ---

# Get English stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords_from_text(text):
    tokens = word_tokenize(text)
    filtered_tokens = [t for t in tokens if t not in stop_words]
    return ' '.join(filtered_tokens)

df_cleaned["Text"] = df_cleaned["Text"].apply(remove_stopwords_from_text)

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)                     # Tokenize
    tokens = [t for t in tokens if t not in stop_words]  # Remove stopwords
    return tokens

df_cleaned["Tokens"] = df_cleaned["Text"].apply(tokenize_and_remove_stopwords)
df_cleaned.head(20)

,Text,Subreddit,Category,TextLength,Tokens
0,man cut rest outdoor inside,soccer,No Slang,11,"[man, cut, rest, outdoor, inside]"
3,doubt even chatgpt would come story stupid bun...,gaming,No Slang,30,"[doubt, even, chatgpt, would, come, story, stu..."
4,feel thats kinda true time also true smaller t...,soccer,No Slang,54,"[feel, thats, kinda, true, time, also, true, s..."
5,im saying experience wrong three degrees ive n...,memes,Internet Slang,24,"[im, saying, experience, wrong, three, degrees..."
6,oppose removal kind history removing historica...,AskReddit,No Slang,111,"[oppose, removal, kind, history, removing, his..."
7,chronic pain dont think ive known moment pain ...,AskReddit,No Slang,15,"[chronic, pain, dont, think, ive, known, momen..."
8,youre twisting words thats ok probably dazed c...,soccer,Offensive Slang,15,"[youre, twisting, words, thats, ok, probably, ..."
10,remember thought chan longest time everyone me...,memes,Internet Slang,30,"[remember, thought, chan, longest, time, every..."
11,yeah keep cortisol levels kids high damage life,funny,No Slang,14,"[yeah, keep, cortisol, levels, kids, high, dam..."
12,give aquafaba egg noodles using aquafaba flour...,NoStupidQuestions,No Slang,26,"[give, aquafaba, egg, noodles, using, aquafaba..."


In [6]:
# Define a custom mapping for categories
category_mapping = {
    "Casual Slang": 0,
    "Internet Slang": 1,
    "Abbreviation": 2,
    "Offensive Slang": 3,
    "No Slang": 4
}

# Apply the mapping to the 'Category' column
df_cleaned["Category_Label"] = df_cleaned["Category"].map(category_mapping)

# Check the results
print(df_cleaned[["Category", "Category_Label"]].head(10))

           Category  Category_Label
0          No Slang               4
3          No Slang               4
4          No Slang               4
5    Internet Slang               1
6          No Slang               4
7          No Slang               4
8   Offensive Slang               3
10   Internet Slang               1
11         No Slang               4
12         No Slang               4


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def filter_tokens(tokens):
    return [t for t in tokens if len(t) > 2]  # remove "aa", "uh", etc.

df_cleaned["Tokens"] = df_cleaned["Tokens"].apply(filter_tokens)

# Join tokens back into text
df_cleaned["Processed_Text"] = df_cleaned["Tokens"].apply(lambda tokens: " ".join(tokens))

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the processed text
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cleaned["Processed_Text"])

# Convert to DataFrame (optional, for inspection or debugging)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Show result
tfidf_df.head()

,aaa,aaaaaand,aaaaahhh,aaaaalmost,aaaah,aaahh,aave,abandon,abandoned,abc,...,zone,zoned,zones,zoolander,zoological,zoom,zoomers,zorro,zotero,zyns
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Features and labels
X = tfidf_matrix
y = df_cleaned["Category_Label"]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=category_mapping.keys()))


Accuracy: 0.6631205673758865

Classification Report:
                  precision    recall  f1-score   support

   Casual Slang       0.50      0.02      0.05        81
 Internet Slang       0.79      0.17      0.28       162
   Abbreviation       0.00      0.00      0.00        65
Offensive Slang       0.81      0.15      0.25       115
       No Slang       0.66      1.00      0.79       705

       accuracy                           0.66      1128
      macro avg       0.55      0.27      0.27      1128
   weighted avg       0.64      0.66      0.56      1128



c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\personal\Code\S6\NLP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
df_cleaned[["Text", "Category", "Category_Label", "Subreddit"]].head()

,Text,Category,Category_Label,Subreddit
0,man cut rest outdoor inside,No Slang,0,soccer
3,doubt even chatgpt would come story stupid bun...,No Slang,0,gaming
4,feel thats kinda true time also true smaller t...,No Slang,0,soccer
5,im saying experience wrong three degrees ive n...,No Slang,0,memes
6,oppose removal kind history removing historica...,No Slang,0,AskReddit


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join tokens back into text
df_cleaned["Processed_Text"] = df_cleaned["Tokens"].apply(lambda tokens: " ".join(tokens))

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the processed text
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cleaned["Processed_Text"])

# Convert to DataFrame (optional, for inspection or debugging)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Show result
print(tfidf_df.head())


    aa  aaa  aaaaaand  aaaaahhh  aaaaalmost  aaaah  aaahh  aave   ab  abandon  \
0  0.0  0.0       0.0       0.0         0.0    0.0    0.0   0.0  0.0      0.0   
1  0.0  0.0       0.0       0.0         0.0    0.0    0.0   0.0  0.0      0.0   
2  0.0  0.0       0.0       0.0         0.0    0.0    0.0   0.0  0.0      0.0   
3  0.0  0.0       0.0       0.0         0.0    0.0    0.0   0.0  0.0      0.0   
4  0.0  0.0       0.0       0.0         0.0    0.0    0.0   0.0  0.0      0.0   

   ...  zone  zoned  zones  zoolander  zoological  zoom  zoomers  zorro  \
0  ...   0.0    0.0    0.0        0.0         0.0   0.0      0.0    0.0   
1  ...   0.0    0.0    0.0        0.0         0.0   0.0      0.0    0.0   
2  ...   0.0    0.0    0.0        0.0         0.0   0.0      0.0    0.0   
3  ...   0.0    0.0    0.0        0.0         0.0   0.0      0.0    0.0   
4  ...   0.0    0.0    0.0        0.0         0.0   0.0      0.0    0.0   

   zotero  zyns  
0     0.0   0.0  
1     0.0   0.0  
2     0.

In [9]:
print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 37502 stored elements and shape (4705, 1000)>
  Coords	Values
  (0, 520)	0.42495060976260296
  (0, 178)	0.5418211382110527
  (0, 713)	0.5077675469892485
  (0, 420)	0.5177054680748433
  (1, 221)	0.40340497816213206
  (1, 135)	0.3285038045027158
  (1, 824)	0.3445320143757628
  (1, 828)	0.3644599032291706
  (1, 89)	0.39350058092725876
  (1, 603)	0.4294566796537136
  (1, 618)	0.3722104205155084
  (2, 281)	0.15665020154456968
  (2, 864)	0.13378343769126438
  (2, 443)	0.21238926104568134
  (2, 892)	0.37958882095320634
  (2, 879)	0.12337948039260886
  (2, 781)	0.45667598946466476
  (2, 851)	0.2087570056034184
  (2, 62)	0.23758627260177828
  (2, 638)	0.15794153382898216
  (2, 302)	0.20247155357452537
  (2, 837)	0.15983472937903323
  (2, 121)	0.20547123087902705
  (2, 81)	0.22273522919484995
  (2, 641)	0.360085691388875
  :	:
  (4701, 705)	0.45599523133909153
  (4701, 445)	0.2738884631761592
  (4702, 390)	0.42833261890923635
  (4702,